In [5]:
# Import necessary libraries
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Function to load research articles dataset
def load_data():
    df = pd.read_csv('data/research_articles.csv')
    return df['TITLE'] + ' ' + df['ABSTRACT']
    
# Function to vectorize text data using TfidfVectorizer
def vectorize_text_tfidf(text):
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    X_vec = vectorizer.fit_transform(text)
    return X_vec, vectorizer
    
# Function to train a Latent Dirichlet Allocation (LDA) model
def train_lda_model(X_vec, num_topics):
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X_vec)
    return lda_model
    
# Function to display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        topics[f"Topic {topic_idx+1}"] = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
    return topics
    
# Main function for Topic Modeling
def main_topic_modeling(text, num_topics=5, num_top_words=10):
    # Step 1: Vectorize text data using TfidfVectorizer
    X_vec, vectorizer = vectorize_text_tfidf(text)
    # Step 2: Train a Latent Dirichlet Allocation (LDA) model
    lda_model = train_lda_model(X_vec, num_topics)
    # Step 3: Display the top words for each topic
    feature_names = vectorizer.get_feature_names_out()
    topics = display_topics(lda_model, feature_names, num_top_words)
    # Display the topics
    print(f"\nTop {num_top_words} words for each topic:")
    for topic, words in topics.items():
        print(f"{topic}: {', '.join(words)}")

if __name__ == "__main__":
    text_data = load_data()
    main_topic_modeling(text_data, num_topics=5, num_top_words=10)


Top 10 words for each topic:
Topic 1: quantum, energy, spin, model, magnetic, phase, field, time, temperature, wave
Topic 2: learning, data, model, network, networks, based, algorithm, models, neural, problem
Topic 3: mn, doping, floquet, fese, t_c, soc, kitaev, semimetals, mos2, verma
Topic 4: qa, nmf, hedging, opioid, password, gerrymandering, hashtags, triad, fuzzing, sequent
Topic 5: mathbb, prove, group, mathcal, finite, groups, theorem, spaces, algebra, space


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Toy dataset
sentences = [
    'The cat sat on the',
    'In the middle of the',
    'She walked into the',
    'He looked at her and said,'
]

# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max(len(seq) for seq in input_sequences)
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = padded_sequences[:, :-1], padded_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Generate text by predicting the next word
seed_text = "He looked at her and said,"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted_index = np.argmax(model.predict(token_list), axis=-1)
    predicted_word = tokenizer.index_word[predicted_index[0]]
    seed_text += " " + predicted_word

print(seed_text)

ModuleNotFoundError: No module named 'tensorflow'

ModuleNotFoundError: No module named 'tensorflow'